In [23]:
!pip install -i https://test.pypi.org/simple/ nsc
    
from nsc.distributions import MultivariateCoupledNormal as MVCN
from nsc.math.entropy_prob import *
from nsc.math.entropy_norm import *
import numpy as np
from scipy.stats import multivariate_normal

Looking in indexes: https://test.pypi.org/simple/


In [29]:
mu_1 = np.array([[0, 0]])
mu_2 = np.array([[0, 0]])

sigma_diag_1 = np.array([[1, 1]])
sigma_diag_2 = np.array([[1.5, 1.5]])

mvn_1 = MVCN(mu_1, np.sqrt(sigma_diag_1))
mvn_2 = MVCN(mu_2, np.sqrt(sigma_diag_2))

In [30]:
true_kl = np.log(np.linalg.det(np.diag(sigma_diag_2[0]))).reshape(-1,) - np.log(np.linalg.det(np.diag(sigma_diag_1[0]))).reshape(-1,)
true_kl -= mu_1.shape[1]
true_kl += np.trace(np.matmul(np.linalg.inv(np.diag(sigma_diag_2[0])), np.diag(sigma_diag_1[0]))).reshape(-1,)
true_kl += np.matmul(np.matmul((mu_2 - mu_1), np.linalg.inv(np.diag(sigma_diag_2[0]))), (mu_2 - mu_1).T).reshape(-1,)
true_kl *= 0.5
true_kl

array([0.07213177])

In [31]:
coupled_kl_divergence_norm(mvn_1, mvn_2, kappa=0, alpha=2, n=10000, seed=1)

0.07210883242226096

In [32]:
n = 10000

samples_1 = mvn_1.sample_n(n)

probs_1 = mvn_1.prob(samples_1).reshape(-1,)
probs_2 = mvn_2.prob(samples_1).reshape(-1,)
    
#coupled_kl_divergence_prob(probs_1, probs_2) / n
np.mean(np.log(probs_1) - np.log(probs_2))

0.0725112546875283

In [28]:
scipy_mvn_1 = multivariate_normal(mu_1[0], sigma_diag_1[0])
scipy_mvn_2 = multivariate_normal(mu_2[0], sigma_diag_2[0])

np.allclose(scipy_mvn_1.pdf(samples_1), probs_1) & np.allclose(scipy_mvn_2.pdf(samples_1), probs_2)

True